# RNA MULTICAPA DE 3 CAPAS
 - capa de entrada
 - 1 capa oculta
 - capa de salida)

## importar librerias

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

## funcion de activacion

In [ ]:
def func_activacion(n):
  return 1/(1+math.exp(-n))   

In [ ]:
func_activacion_vect = np.vectorize(func_activacion)

## RNA

In [ ]:
def neurona(pesos,entrada, bias):
  prod_punto = np.dot(pesos,entrada)+bias # z = W.X + b
  return func_activacion_vect(prod_punto) # f(z)

## RNA Multicapa

In [ ]:
def red_neuronal_multicapa(input, pesosc1, biasc1, pesoscs, biascs):
  # Capa entrada 
  entrada_x = input
  # Capa oculta 1
  salida_c1 = neurona(pesosc1, entrada_x, biasc1)
  # Capa de SALIDA
  output_cs = neurona(pesoscs, salida_c1, biascs)
  # Retornar resultados  
  return salida_c1, output_cs

## derivada de la funcion

In [ ]:
def derivada_f(salida_y): # derivada de la funcion sigmoide
  return salida_y*(1-salida_y)

In [ ]:
def adjust_output(output_wish,size,ExistZero):
  SD = np.zeros(size) # vector con el nro de neuronas 
  if ExistZero == True:
    SD[output_wish] = 1
  else:
    SD[output_wish - 1] = 1
  return SD

## backpropagation

In [ ]:
def propagacion_hacia_atras(entradas,salidas_deseadas,alfa,neuronas_c1,neuronas_cs,error=0.005,epochs=1000):
  flag = True if 0 in salidas_deseadas.unique() else False
  # longitud de la entrada(caracteristicas)
  n_entradas = entradas.shape[1]

  # Capa oculta 1
  pesosc1 = 2*np.random.rand(neuronas_c1,n_entradas) -1   # filas: nro neuronas x capa, col = nro inputs que recibe la neurona
  biasc1 = 2*np.random.rand(neuronas_c1)-1
  # Capa de salida
  pesoscs = 2*np.random.rand(neuronas_cs,neuronas_c1) - 1 # filas: nro neuronas x capa, col = nro inputs que recibe la neurona
  biascs = 2*np.random.rand(neuronas_cs) - 1

  # Iterar el algoritmo(nro de epocas)
  for epoch in range(0,epochs):
    # Evaluar los patrones(para c/entrada y salida) respectivamente
    for entrada_i,salida_deseada_i in zip(entradas,salidas_deseadas):
      # Predecir usando la red neuronal
      salida_c1, output_cs = red_neuronal_multicapa(entrada_i, pesosc1, biasc1, pesoscs, biascs)

      # Calcular los errores producidos en cada capa de forma invertida(retropropagacion) y actualizar pesos
      # ----------------------------------------------------------------------------------------------------
      salida_deseada = adjust_output(salida_deseada_i,neuronas_cs,flag)
      # Calcular Δ para la CAPA de SALIDA(cs): 
      # Δ Salida_Y = f'(Salida_Y) * (salidaY_Deseada - Salida_Y) 
      # F' de la func sigmoide: f'(Salida_Y) = Salida_Y * (1 - Salida_Y)
      delta_cs = derivada_f(output_cs)*(salida_deseada - output_cs)
      peso_anterior_cs = pesoscs
      
      # Actualizar pesos que llegan a la capa de SALIDA
      # W = W_ingresan_a_las_ncs + alfa*Salida_y(capa anterior)*Δ_cs
      pesoscs = pesoscs + alfa * delta_cs.reshape(1,-1).T * salida_c1.reshape(1,-1) 
      biascs = biascs + alfa * delta_cs*1   # b = b + alfa*Δ

      # Calcular Δ para la CAPA OCULTA 1 (pesosc1):      
      # Δ Salida_Y = f'(Salida_Y) * (W_entran_c1 * ΔSalida_CS)
      delta_c1 = derivada_f(salida_c1) * np.dot(peso_anterior_cs.T, delta_cs) 
      
      # Actualizar pesos  que entran a las neuronas de la capa oculta 1
      # W = W_ingresan_a_las_ncs + alfa*Salida_y(capa anterior)*Δ_c1
      pesosc1 = pesosc1 + alfa * (delta_c1.reshape(1,-1)*entrada_i.reshape(1,-1).T).T 
      biasc1 = biasc1 + alfa * delta_c1 * 1  # b = b + alfa*Δ_c1
      
  return pesosc1,biasc1,pesoscs,biascs 

## fit RNA

In [ ]:
def fit_rna(entradas,salidas_deseadas,alfa,neuronas_c1,neuronas_cs,error,epochs):
  pesosc1,biasc1,pesoscs,biascs = propagacion_hacia_atras(entradas,salidas_deseadas,alfa,neuronas_c1,neuronas_cs,error,epochs)
  return pesosc1,biasc1,pesoscs,biascs


## predict RNA

In [ ]:
def predict_rna(x_test, valores_obtenidos):
  pesosc1,biasc1,pesoscs,biascs = valores_obtenidos[0],valores_obtenidos[1],valores_obtenidos[2],valores_obtenidos[3]
  y_predict_list = []
  for entrada in x_test:
    salida_c1, output_cs = red_neuronal_multicapa(entrada, pesosc1,biasc1,pesoscs,biascs)
    y_predict_list.append(output_cs)
  return y_predict_list

# DETECCION DE DIGITOS

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix

# digitos

In [ ]:
df = pd.read_csv("iris.csv")
df.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [ ]:
df['Species'].unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [ ]:
def Categ_Numerico(datos):
    # Conversión de un tipo de datos al tipo categorico
    datos = datos.astype("category")
    # Obtención del código de la categoría
    return datos.astype("category").cat.codes

In [ ]:
df['Species'] = Categ_Numerico(df['Species'])
df['Species'].unique()

array([0, 1, 2], dtype=int8)

In [ ]:
# =========== NORMALIZANDO ================== 
X = df.drop("Species", axis=1)
Y = df["Species"]

scaler = preprocessing.StandardScaler()
scaler.fit(X)
# =========== TRANSFORMANDO ================= 
X = scaler.transform(X)

In [ ]:
# =========== SEPARACION DE DATOS =========== 
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
# =========== FIT ===========================  
pesosc1,biasc1,pesoscs,biascs = fit_rna(X_train,y_train,0.1,12,3,0.005,100)

In [ ]:
# =========== PREDICT ======================= 
pesos_bias_optimos = [pesosc1,biasc1,pesoscs,biascs]   # recuperar los pesos y bias optimos
valores_pred = predict_rna(X_test, pesos_bias_optimos) # predecir

In [ ]:
# =========== ajustar salida ================ 
def ajuste(valores_pred):
  result_pred = []
  for vector in valores_pred:
    indice = np.argmax(vector)
    result_pred.append(indice)
  return result_pred

In [ ]:
y_pred = pd.Series(ajuste(valores_pred)) # ajustar valores de salida(obtener los resultads)

In [ ]:
# =========== METRICAS DE EVALUACION ========= 
print(classification_report(y_test,y_pred ))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.89      0.94         9
           2       0.92      1.00      0.96        11

    accuracy                           0.97        30
   macro avg       0.97      0.96      0.97        30
weighted avg       0.97      0.97      0.97        30



In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score, f1_score

valores_pred = predict_rna(X_train, pesos_bias_optimos)
y_train_pred = pd.Series(ajuste(valores_pred))

# verificando aprendizaje
print("Aprendizaje - Entrenamiento")
print(balanced_accuracy_score(y_train, y_train_pred))
print(f1_score(y_train, y_train_pred, average="weighted"))
print('\n')
#verificando generalización
print("Prueba - Generalización")
print(balanced_accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average="weighted"))

Aprendizaje - Entrenamiento
0.9751928288513655
0.975003906860447


Prueba - Generalización
0.9629629629629629
0.9664109121909632


## prediccion

In [ ]:
# [0:'setosa', 1:'versicolor', 2:'virginica']

In [ ]:
# datos de entrada
entrada = [6.2, 	3.1, 	5.3, 	2.2]

In [ ]:
entrada_df = np.array([entrada]) # convertir a array de numpy y lista de listas
# NORMALIZANDO LOS DATOS DE ENTRADA
entrada_df = scaler.transform(entrada_df)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
result = np.argmax(predict_rna(entrada_df, pesos_bias_optimos))
if result == 0:
  print("setosa")
elif result == 1 :
  print("versicolor")
else:
  print("virginica")

virginica
